# Install Library

In [1]:
%%capture
%%time
!pip install tqdm
!pip install sastrawi
!pip install swifter

KeyboardInterrupt: 

# Import Library

In [ ]:
# from google.colab import drive
import pandas as pd
import re
import string
from tqdm import tqdm

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter


import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

# detokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer



[nltk_data] Downloading package punkt to /Users/alfinst28/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alfinst28/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Load Data

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
# drive.mount('/content/drive/', force_remount=True)
raw_dataset = pd.read_csv("contoh_data")

raw_dataset

,id,keyword,datetime,tweet_id,text,username,label_1,bahasa
0,0,kontol,2023-02-22 23:59:03+00:00,1628544930632327168,@tanyakanrl Polisinya kontol abiez,CnfusedSkelly,1,Indonesia
1,1,kontol,2023-02-22 23:58:34+00:00,1628544810121588744,pagi pagi udah kaya kontol aja https://t.co/tn...,butterflyrhm,0,Indonesia
2,2,kontol,2023-02-22 23:57:53+00:00,1628544639392448513,@salisaucee @Arselowono @tanyakanrl Wkwk KONTO...,potophile,1,Indonesia
3,3,kontol,2023-02-22 23:57:45+00:00,1628544605171093504,kontol,udzzlee,0,Indonesia
4,4,kontol,2023-02-22 23:57:33+00:00,1628544554680082433,kontol kontol kontol KONTOL KOnTOLLLLLL,arthifer,0,Indonesia
...,...,...,...,...,...,...,...,...
2900,2104,lonte,2023-03-01 02:08:46+00:00,1630751904308555777,@indocringe_ Sumpah kalau Gacoan ini &amp; mix...,SnowyLand95,1,indonesia
2901,2106,lonte,2023-03-01 01:42:03+00:00,1630745181426450432,@opposite6892 Lu anak LONTE ya,Yha2n3,1,indonesia
2902,2107,lonte,2023-03-01 01:39:27+00:00,1630744525139492864,@GYOHANS_ ya gmn ya bu anak lo aja kek lonte gitu,reinanbraun,1,indonesia
2903,2113,lonte,2023-03-01 00:41:39+00:00,1630729979058352128,"@puji_setyanto @CH4__15R Eh TOLOL bocah cacat,...",septa_ringga,1,indonesia


# Preprocessing

## Data Cleaning

In [ ]:
#CHECKING MISSING VALUES
raw_dataset.isnull().sum()

id          0
keyword     0
datetime    0
tweet_id    0
text        0
username    0
label_1     0
bahasa      0
dtype: int64

In [ ]:
# Preprocessing data
def cleaning_text(text):
    # Hapus tagging username
    text = re.sub(r'@\w+', ' ', text)
    text = re.sub(r'@\S+', ' ', text)
    text = re.sub(r'#\S+', ' ', text)

    # hapus emoji
    regrex_pattern = re.compile(pattern = "["
      u"\U00002700-\U000027BF"  # Dingbats
      u"\U0001F600-\U0001F64F"  # Emoticons
      u"\U00002600-\U000026FF"  # Miscellaneous Symbols
      u"\U0001F300-\U0001F5FF"  # Miscellaneous Symbols And Pictographs
      u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
      u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
      u"\U0001F680-\U0001F6FF"  # Transport and Map Symbols
      "]+", flags = re.UNICODE)
    text = regrex_pattern.sub(r' ',text)

    # Hapus tanda baca, angka, dan karakter khusus
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\\x\S+', ' ', text)
    text = re.sub(r'\\u\S+', ' ', text)

    for sp in string.punctuation:
      text = text.replace(sp, " ")
    text = re.sub(r"\d+","",text)
    text = text.replace('/\s\s+/g', ' ')

    # Hapus kata RT
    text = re.sub(r'\bRT\b', ' ', text)
    # Hapus URL
    text = re.sub(r'((www.\.[^\s]+)|(https?://[^\s]+))', ' ', text)
    text = re.sub(r'http\S+', ' ', text)

    # Hapus text yang menggabungkan karakter yang sama yang berulang berturut-turut dalam suatu teks
    text = re.sub(r'(.)\1+', r'\1\1', text)
    text = re.sub(r'\\n', ' ', text)

    # Hapus whitespace berlebih
    text = text.strip()
    text = re.sub('\s+', ' ', text)

    return text

# Lakukan preprocessing pada kolom 'text'
tqdm.pandas(desc="Processing Cleaning Text: ")
raw_dataset['clean'] = raw_dataset['text'].apply(cleaning_text)

raw_dataset

,id,keyword,datetime,tweet_id,text,username,label_1,bahasa,clean
0,0,kontol,2023-02-22 23:59:03+00:00,1628544930632327168,@tanyakanrl Polisinya kontol abiez,CnfusedSkelly,1,Indonesia,Polisinya kontol abiez
1,1,kontol,2023-02-22 23:58:34+00:00,1628544810121588744,pagi pagi udah kaya kontol aja https://t.co/tn...,butterflyrhm,0,Indonesia,pagi pagi udah kaya kontol aja t co tnyAKwuJl
2,2,kontol,2023-02-22 23:57:53+00:00,1628544639392448513,@salisaucee @Arselowono @tanyakanrl Wkwk KONTO...,potophile,1,Indonesia,Wkwk KONTOL Konyol dan tolol
3,3,kontol,2023-02-22 23:57:45+00:00,1628544605171093504,kontol,udzzlee,0,Indonesia,kontol
4,4,kontol,2023-02-22 23:57:33+00:00,1628544554680082433,kontol kontol kontol KONTOL KOnTOLLLLLL,arthifer,0,Indonesia,kontol kontol kontol KONTOL KOnTOLL
...,...,...,...,...,...,...,...,...,...
2900,2104,lonte,2023-03-01 02:08:46+00:00,1630751904308555777,@indocringe_ Sumpah kalau Gacoan ini &amp; mix...,SnowyLand95,1,indonesia,Sumpah kalau Gacoan ini amp mixue bukan settin...
2901,2106,lonte,2023-03-01 01:42:03+00:00,1630745181426450432,@opposite6892 Lu anak LONTE ya,Yha2n3,1,indonesia,Lu anak LONTE ya
2902,2107,lonte,2023-03-01 01:39:27+00:00,1630744525139492864,@GYOHANS_ ya gmn ya bu anak lo aja kek lonte gitu,reinanbraun,1,indonesia,ya gmn ya bu anak lo aja kek lonte gitu
2903,2113,lonte,2023-03-01 00:41:39+00:00,1630729979058352128,"@puji_setyanto @CH4__15R Eh TOLOL bocah cacat,...",septa_ringga,1,indonesia,Eh TOLOL bocah cacat di situ ada perangkat des...


In [ ]:
raw_dataset['clean'][995]

'Opini lo jelek kontol gak heran sih ngestan aja sama monyet'

## Case Folding

In [ ]:
# Uppercase to lowercase
def case_folding_text(text):
  return str(text).lower()

tqdm.pandas(desc="Processing Case Folding: ")
raw_dataset['clean'] = raw_dataset['clean'].progress_apply(case_folding_text)
raw_dataset.head()

Processing Case Folding: 100%|██████████| 2905/2905 [00:00<00:00, 487397.62it/s]


,id,keyword,datetime,tweet_id,text,username,label_1,bahasa,clean
0,0,kontol,2023-02-22 23:59:03+00:00,1628544930632327168,@tanyakanrl Polisinya kontol abiez,CnfusedSkelly,1,Indonesia,polisinya kontol abiez
1,1,kontol,2023-02-22 23:58:34+00:00,1628544810121588744,pagi pagi udah kaya kontol aja https://t.co/tn...,butterflyrhm,0,Indonesia,pagi pagi udah kaya kontol aja t co tnyakwujl
2,2,kontol,2023-02-22 23:57:53+00:00,1628544639392448513,@salisaucee @Arselowono @tanyakanrl Wkwk KONTO...,potophile,1,Indonesia,wkwk kontol konyol dan tolol
3,3,kontol,2023-02-22 23:57:45+00:00,1628544605171093504,kontol,udzzlee,0,Indonesia,kontol
4,4,kontol,2023-02-22 23:57:33+00:00,1628544554680082433,kontol kontol kontol KONTOL KOnTOLLLLLL,arthifer,0,Indonesia,kontol kontol kontol kontol kontoll


## Tokenization

In [ ]:
def tokenizing_text(data):
    return nltk.tokenize.word_tokenize(data)

tqdm.pandas(desc="Processing tokenize: ")
raw_dataset['tokens'] = raw_dataset['clean'].progress_apply(tokenizing_text)
raw_dataset.head()

Processing tokenize: 100%|████████████████| 2905/2905 [00:00<00:00, 9060.97it/s]


,id,keyword,datetime,tweet_id,text,username,label_1,bahasa,clean,tokens
0,0,kontol,2023-02-22 23:59:03+00:00,1628544930632327168,@tanyakanrl Polisinya kontol abiez,CnfusedSkelly,1,Indonesia,polisinya kontol abiez,"[polisinya, kontol, abiez]"
1,1,kontol,2023-02-22 23:58:34+00:00,1628544810121588744,pagi pagi udah kaya kontol aja https://t.co/tn...,butterflyrhm,0,Indonesia,pagi pagi udah kaya kontol aja t co tnyakwujl,"[pagi, pagi, udah, kaya, kontol, aja, t, co, t..."
2,2,kontol,2023-02-22 23:57:53+00:00,1628544639392448513,@salisaucee @Arselowono @tanyakanrl Wkwk KONTO...,potophile,1,Indonesia,wkwk kontol konyol dan tolol,"[wkwk, kontol, konyol, dan, tolol]"
3,3,kontol,2023-02-22 23:57:45+00:00,1628544605171093504,kontol,udzzlee,0,Indonesia,kontol,[kontol]
4,4,kontol,2023-02-22 23:57:33+00:00,1628544554680082433,kontol kontol kontol KONTOL KOnTOLLLLLL,arthifer,0,Indonesia,kontol kontol kontol kontol kontoll,"[kontol, kontol, kontol, kontol, kontoll]"


## Normalisasi

In [ ]:
dict_normalisasi = pd.read_csv('dataset/kamus_alay.csv')
dict_normalisasi

NameError: name 'pd' is not defined

In [ ]:
dict_norm = dict(dict_normalisasi.values)

def normalized_term(document):
    filtered_document = document.copy()
    for idx, term in enumerate(filtered_document):
      if term in dict_norm:
          filtered_document[idx] = dict_norm[term]
    return filtered_document

tqdm.pandas(desc="Processing Normalisasi: ")
raw_dataset['tokens_normalized'] = raw_dataset['tokens'].progress_apply(normalized_term)
raw_dataset

Processing Normalisasi: 100%|███████████| 2905/2905 [00:00<00:00, 309965.99it/s]


,id,keyword,datetime,tweet_id,text,username,label_1,bahasa,clean,tokens,tokens_normalized
0,0,kontol,2023-02-22 23:59:03+00:00,1628544930632327168,@tanyakanrl Polisinya kontol abiez,CnfusedSkelly,1,Indonesia,polisinya kontol abiez,"[polisinya, kontol, abiez]","[polisinya, kontol, habis]"
1,1,kontol,2023-02-22 23:58:34+00:00,1628544810121588744,pagi pagi udah kaya kontol aja https://t.co/tn...,butterflyrhm,0,Indonesia,pagi pagi udah kaya kontol aja t co tnyakwujl,"[pagi, pagi, udah, kaya, kontol, aja, t, co, t...","[pagi, pagi, sudah, kayak, kontol, saja, t, co..."
2,2,kontol,2023-02-22 23:57:53+00:00,1628544639392448513,@salisaucee @Arselowono @tanyakanrl Wkwk KONTO...,potophile,1,Indonesia,wkwk kontol konyol dan tolol,"[wkwk, kontol, konyol, dan, tolol]","[wkwk, kontol, konyol, dan, tolol]"
3,3,kontol,2023-02-22 23:57:45+00:00,1628544605171093504,kontol,udzzlee,0,Indonesia,kontol,[kontol],[kontol]
4,4,kontol,2023-02-22 23:57:33+00:00,1628544554680082433,kontol kontol kontol KONTOL KOnTOLLLLLL,arthifer,0,Indonesia,kontol kontol kontol kontol kontoll,"[kontol, kontol, kontol, kontol, kontoll]","[kontol, kontol, kontol, kontol, kontoll]"
...,...,...,...,...,...,...,...,...,...,...,...
2900,2104,lonte,2023-03-01 02:08:46+00:00,1630751904308555777,@indocringe_ Sumpah kalau Gacoan ini &amp; mix...,SnowyLand95,1,indonesia,sumpah kalau gacoan ini amp mixue bukan settin...,"[sumpah, kalau, gacoan, ini, amp, mixue, bukan...","[sumpah, kalau, gacoan, ini, amp, mixue, bukan..."
2901,2106,lonte,2023-03-01 01:42:03+00:00,1630745181426450432,@opposite6892 Lu anak LONTE ya,Yha2n3,1,indonesia,lu anak lonte ya,"[lu, anak, lonte, ya]","[kamu, anak, lonte, ya]"
2902,2107,lonte,2023-03-01 01:39:27+00:00,1630744525139492864,@GYOHANS_ ya gmn ya bu anak lo aja kek lonte gitu,reinanbraun,1,indonesia,ya gmn ya bu anak lo aja kek lonte gitu,"[ya, gmn, ya, bu, anak, lo, aja, kek, lonte, g...","[ya, bagaimana, ya, bu, anak, kamu, saja, kaya..."
2903,2113,lonte,2023-03-01 00:41:39+00:00,1630729979058352128,"@puji_setyanto @CH4__15R Eh TOLOL bocah cacat,...",septa_ringga,1,indonesia,eh tolol bocah cacat di situ ada perangkat des...,"[eh, tolol, bocah, cacat, di, situ, ada, peran...","[eh, tolol, bocah, cacat, di, situ, ada, peran..."


## Stopword

In [ ]:
# Preprocessing data
def stopword_text(tokens):
    stopword = stopwords.words('indonesian')
    filtered_tokens = [token for token in tokens if token not in stopword]
    return filtered_tokens

tqdm.pandas(desc="Processing Stopword: ")
raw_dataset['tokens_normalized_stopwords'] = raw_dataset['tokens_normalized'].progress_apply(stopword_text)

raw_dataset.head()

Processing Stopword: 100%|████████████████| 2905/2905 [00:00<00:00, 5757.75it/s]


,id,keyword,datetime,tweet_id,text,username,label_1,bahasa,clean,tokens,tokens_normalized,tokens_normalized_stopwords
0,0,kontol,2023-02-22 23:59:03+00:00,1628544930632327168,@tanyakanrl Polisinya kontol abiez,CnfusedSkelly,1,Indonesia,polisinya kontol abiez,"[polisinya, kontol, abiez]","[polisinya, kontol, habis]","[polisinya, kontol, habis]"
1,1,kontol,2023-02-22 23:58:34+00:00,1628544810121588744,pagi pagi udah kaya kontol aja https://t.co/tn...,butterflyrhm,0,Indonesia,pagi pagi udah kaya kontol aja t co tnyakwujl,"[pagi, pagi, udah, kaya, kontol, aja, t, co, t...","[pagi, pagi, sudah, kayak, kontol, saja, t, co...","[pagi, pagi, kayak, kontol, t, cowok, tnyakwujl]"
2,2,kontol,2023-02-22 23:57:53+00:00,1628544639392448513,@salisaucee @Arselowono @tanyakanrl Wkwk KONTO...,potophile,1,Indonesia,wkwk kontol konyol dan tolol,"[wkwk, kontol, konyol, dan, tolol]","[wkwk, kontol, konyol, dan, tolol]","[wkwk, kontol, konyol, tolol]"
3,3,kontol,2023-02-22 23:57:45+00:00,1628544605171093504,kontol,udzzlee,0,Indonesia,kontol,[kontol],[kontol],[kontol]
4,4,kontol,2023-02-22 23:57:33+00:00,1628544554680082433,kontol kontol kontol KONTOL KOnTOLLLLLL,arthifer,0,Indonesia,kontol kontol kontol kontol kontoll,"[kontol, kontol, kontol, kontol, kontoll]","[kontol, kontol, kontol, kontol, kontoll]","[kontol, kontol, kontol, kontol, kontoll]"


## Stemming

In [ ]:
# Preprocessing data
def stemming_text(tokens):
    # Stemming teks dengan Sastrawi
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    stems = [stemmer.stem(token) for token in tokens]
    return stems

tqdm.pandas(desc="Processing Stemming: ")
raw_dataset['tokens_normalized_stopwords_stemmed'] = raw_dataset['tokens_normalized_stopwords'].progress_apply(stemming_text)

raw_dataset.head()

Processing Stemming: 100%|██████████████████| 2905/2905 [05:02<00:00,  9.60it/s]


,id,keyword,datetime,tweet_id,text,username,label_1,bahasa,clean,tokens,tokens_normalized,tokens_normalized_stopwords,tokens_normalized_stopwords_stemmed
0,0,kontol,2023-02-22 23:59:03+00:00,1628544930632327168,@tanyakanrl Polisinya kontol abiez,CnfusedSkelly,1,Indonesia,polisinya kontol abiez,"[polisinya, kontol, abiez]","[polisinya, kontol, habis]","[polisinya, kontol, habis]","[polisi, kontol, habis]"
1,1,kontol,2023-02-22 23:58:34+00:00,1628544810121588744,pagi pagi udah kaya kontol aja https://t.co/tn...,butterflyrhm,0,Indonesia,pagi pagi udah kaya kontol aja t co tnyakwujl,"[pagi, pagi, udah, kaya, kontol, aja, t, co, t...","[pagi, pagi, sudah, kayak, kontol, saja, t, co...","[pagi, pagi, kayak, kontol, t, cowok, tnyakwujl]","[pagi, pagi, kayak, kontol, t, cowok, tnyakwujl]"
2,2,kontol,2023-02-22 23:57:53+00:00,1628544639392448513,@salisaucee @Arselowono @tanyakanrl Wkwk KONTO...,potophile,1,Indonesia,wkwk kontol konyol dan tolol,"[wkwk, kontol, konyol, dan, tolol]","[wkwk, kontol, konyol, dan, tolol]","[wkwk, kontol, konyol, tolol]","[wkwk, kontol, konyol, tolol]"
3,3,kontol,2023-02-22 23:57:45+00:00,1628544605171093504,kontol,udzzlee,0,Indonesia,kontol,[kontol],[kontol],[kontol],[kontol]
4,4,kontol,2023-02-22 23:57:33+00:00,1628544554680082433,kontol kontol kontol KONTOL KOnTOLLLLLL,arthifer,0,Indonesia,kontol kontol kontol kontol kontoll,"[kontol, kontol, kontol, kontol, kontoll]","[kontol, kontol, kontol, kontol, kontoll]","[kontol, kontol, kontol, kontol, kontoll]","[kontol, kontol, kontol, kontol, kontoll]"


In [ ]:
raw_dataset

,id,keyword,datetime,tweet_id,text,username,label_1,bahasa,clean,tokens,tokens_normalized,tokens_normalized_stopwords,tokens_normalized_stopwords_stemmed
0,0,kontol,2023-02-22 23:59:03+00:00,1628544930632327168,@tanyakanrl Polisinya kontol abiez,CnfusedSkelly,1,Indonesia,polisinya kontol abiez,"[polisinya, kontol, abiez]","[polisinya, kontol, habis]","[polisinya, kontol, habis]","[polisi, kontol, habis]"
1,1,kontol,2023-02-22 23:58:34+00:00,1628544810121588744,pagi pagi udah kaya kontol aja https://t.co/tn...,butterflyrhm,0,Indonesia,pagi pagi udah kaya kontol aja t co tnyakwujl,"[pagi, pagi, udah, kaya, kontol, aja, t, co, t...","[pagi, pagi, sudah, kayak, kontol, saja, t, co...","[pagi, pagi, kayak, kontol, t, cowok, tnyakwujl]","[pagi, pagi, kayak, kontol, t, cowok, tnyakwujl]"
2,2,kontol,2023-02-22 23:57:53+00:00,1628544639392448513,@salisaucee @Arselowono @tanyakanrl Wkwk KONTO...,potophile,1,Indonesia,wkwk kontol konyol dan tolol,"[wkwk, kontol, konyol, dan, tolol]","[wkwk, kontol, konyol, dan, tolol]","[wkwk, kontol, konyol, tolol]","[wkwk, kontol, konyol, tolol]"
3,3,kontol,2023-02-22 23:57:45+00:00,1628544605171093504,kontol,udzzlee,0,Indonesia,kontol,[kontol],[kontol],[kontol],[kontol]
4,4,kontol,2023-02-22 23:57:33+00:00,1628544554680082433,kontol kontol kontol KONTOL KOnTOLLLLLL,arthifer,0,Indonesia,kontol kontol kontol kontol kontoll,"[kontol, kontol, kontol, kontol, kontoll]","[kontol, kontol, kontol, kontol, kontoll]","[kontol, kontol, kontol, kontol, kontoll]","[kontol, kontol, kontol, kontol, kontoll]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2900,2104,lonte,2023-03-01 02:08:46+00:00,1630751904308555777,@indocringe_ Sumpah kalau Gacoan ini &amp; mix...,SnowyLand95,1,indonesia,sumpah kalau gacoan ini amp mixue bukan settin...,"[sumpah, kalau, gacoan, ini, amp, mixue, bukan...","[sumpah, kalau, gacoan, ini, amp, mixue, bukan...","[sumpah, gacoan, amp, mixue, setingan, kejadia...","[sumpah, gaco, amp, mixue, ting, jadi, gue, si..."
2901,2106,lonte,2023-03-01 01:42:03+00:00,1630745181426450432,@opposite6892 Lu anak LONTE ya,Yha2n3,1,indonesia,lu anak lonte ya,"[lu, anak, lonte, ya]","[kamu, anak, lonte, ya]","[anak, lonte, ya]","[anak, lonte, ya]"
2902,2107,lonte,2023-03-01 01:39:27+00:00,1630744525139492864,@GYOHANS_ ya gmn ya bu anak lo aja kek lonte gitu,reinanbraun,1,indonesia,ya gmn ya bu anak lo aja kek lonte gitu,"[ya, gmn, ya, bu, anak, lo, aja, kek, lonte, g...","[ya, bagaimana, ya, bu, anak, kamu, saja, kaya...","[ya, ya, bu, anak, kayak, lonte]","[ya, ya, bu, anak, kayak, lonte]"
2903,2113,lonte,2023-03-01 00:41:39+00:00,1630729979058352128,"@puji_setyanto @CH4__15R Eh TOLOL bocah cacat,...",septa_ringga,1,indonesia,eh tolol bocah cacat di situ ada perangkat des...,"[eh, tolol, bocah, cacat, di, situ, ada, peran...","[eh, tolol, bocah, cacat, di, situ, ada, peran...","[eh, tolol, bocah, cacat, situ, perangkat, des...","[eh, tolol, bocah, cacat, situ, perangkat, des..."


## Detokenize

In [ ]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

def detokenize(tokens):
  return TreebankWordDetokenizer().detokenize(tokens)

tqdm.pandas(desc="Processing De-tokenize: ")
raw_dataset['detokenize'] = raw_dataset['tokens_normalized_stopwords_stemmed'].progress_apply(detokenize)
raw_dataset.head()

Processing De-tokenize: 100%|████████████| 2905/2905 [00:00<00:00, 22923.45it/s]


,id,keyword,datetime,tweet_id,text,username,label_1,bahasa,clean,tokens,tokens_normalized,tokens_normalized_stopwords,tokens_normalized_stopwords_stemmed,detokenize
0,0,kontol,2023-02-22 23:59:03+00:00,1628544930632327168,@tanyakanrl Polisinya kontol abiez,CnfusedSkelly,1,Indonesia,polisinya kontol abiez,"[polisinya, kontol, abiez]","[polisinya, kontol, habis]","[polisinya, kontol, habis]","[polisi, kontol, habis]",polisi kontol habis
1,1,kontol,2023-02-22 23:58:34+00:00,1628544810121588744,pagi pagi udah kaya kontol aja https://t.co/tn...,butterflyrhm,0,Indonesia,pagi pagi udah kaya kontol aja t co tnyakwujl,"[pagi, pagi, udah, kaya, kontol, aja, t, co, t...","[pagi, pagi, sudah, kayak, kontol, saja, t, co...","[pagi, pagi, kayak, kontol, t, cowok, tnyakwujl]","[pagi, pagi, kayak, kontol, t, cowok, tnyakwujl]",pagi pagi kayak kontol t cowok tnyakwujl
2,2,kontol,2023-02-22 23:57:53+00:00,1628544639392448513,@salisaucee @Arselowono @tanyakanrl Wkwk KONTO...,potophile,1,Indonesia,wkwk kontol konyol dan tolol,"[wkwk, kontol, konyol, dan, tolol]","[wkwk, kontol, konyol, dan, tolol]","[wkwk, kontol, konyol, tolol]","[wkwk, kontol, konyol, tolol]",wkwk kontol konyol tolol
3,3,kontol,2023-02-22 23:57:45+00:00,1628544605171093504,kontol,udzzlee,0,Indonesia,kontol,[kontol],[kontol],[kontol],[kontol],kontol
4,4,kontol,2023-02-22 23:57:33+00:00,1628544554680082433,kontol kontol kontol KONTOL KOnTOLLLLLL,arthifer,0,Indonesia,kontol kontol kontol kontol kontoll,"[kontol, kontol, kontol, kontol, kontoll]","[kontol, kontol, kontol, kontol, kontoll]","[kontol, kontol, kontol, kontol, kontoll]","[kontol, kontol, kontol, kontol, kontoll]",kontol kontol kontol kontol kontoll


In [ ]:
raw_dataset.to_csv('dataset/tweet_after_preprocessing_kntl_bgsd_lonte.csv')

In [ ]:
raw_dataset

,id,keyword,datetime,tweet_id,text,username,label_1,bahasa,clean,tokens,tokens_normalized,tokens_normalized_stopwords,tokens_normalized_stopwords_stemmed,detokenize
0,0,kontol,2023-02-22 23:59:03+00:00,1628544930632327168,@tanyakanrl Polisinya kontol abiez,CnfusedSkelly,1,Indonesia,polisinya kontol abiez,"[polisinya, kontol, abiez]","[polisinya, kontol, habis]","[polisinya, kontol, habis]","[polisi, kontol, habis]",polisi kontol habis
1,1,kontol,2023-02-22 23:58:34+00:00,1628544810121588744,pagi pagi udah kaya kontol aja https://t.co/tn...,butterflyrhm,0,Indonesia,pagi pagi udah kaya kontol aja t co tnyakwujl,"[pagi, pagi, udah, kaya, kontol, aja, t, co, t...","[pagi, pagi, sudah, kayak, kontol, saja, t, co...","[pagi, pagi, kayak, kontol, t, cowok, tnyakwujl]","[pagi, pagi, kayak, kontol, t, cowok, tnyakwujl]",pagi pagi kayak kontol t cowok tnyakwujl
2,2,kontol,2023-02-22 23:57:53+00:00,1628544639392448513,@salisaucee @Arselowono @tanyakanrl Wkwk KONTO...,potophile,1,Indonesia,wkwk kontol konyol dan tolol,"[wkwk, kontol, konyol, dan, tolol]","[wkwk, kontol, konyol, dan, tolol]","[wkwk, kontol, konyol, tolol]","[wkwk, kontol, konyol, tolol]",wkwk kontol konyol tolol
3,3,kontol,2023-02-22 23:57:45+00:00,1628544605171093504,kontol,udzzlee,0,Indonesia,kontol,[kontol],[kontol],[kontol],[kontol],kontol
4,4,kontol,2023-02-22 23:57:33+00:00,1628544554680082433,kontol kontol kontol KONTOL KOnTOLLLLLL,arthifer,0,Indonesia,kontol kontol kontol kontol kontoll,"[kontol, kontol, kontol, kontol, kontoll]","[kontol, kontol, kontol, kontol, kontoll]","[kontol, kontol, kontol, kontol, kontoll]","[kontol, kontol, kontol, kontol, kontoll]",kontol kontol kontol kontol kontoll
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2900,2104,lonte,2023-03-01 02:08:46+00:00,1630751904308555777,@indocringe_ Sumpah kalau Gacoan ini &amp; mix...,SnowyLand95,1,indonesia,sumpah kalau gacoan ini amp mixue bukan settin...,"[sumpah, kalau, gacoan, ini, amp, mixue, bukan...","[sumpah, kalau, gacoan, ini, amp, mixue, bukan...","[sumpah, gacoan, amp, mixue, setingan, kejadia...","[sumpah, gaco, amp, mixue, ting, jadi, gue, si...",sumpah gaco amp mixue ting jadi gue sih bilang...
2901,2106,lonte,2023-03-01 01:42:03+00:00,1630745181426450432,@opposite6892 Lu anak LONTE ya,Yha2n3,1,indonesia,lu anak lonte ya,"[lu, anak, lonte, ya]","[kamu, anak, lonte, ya]","[anak, lonte, ya]","[anak, lonte, ya]",anak lonte ya
2902,2107,lonte,2023-03-01 01:39:27+00:00,1630744525139492864,@GYOHANS_ ya gmn ya bu anak lo aja kek lonte gitu,reinanbraun,1,indonesia,ya gmn ya bu anak lo aja kek lonte gitu,"[ya, gmn, ya, bu, anak, lo, aja, kek, lonte, g...","[ya, bagaimana, ya, bu, anak, kamu, saja, kaya...","[ya, ya, bu, anak, kayak, lonte]","[ya, ya, bu, anak, kayak, lonte]",ya ya bu anak kayak lonte
2903,2113,lonte,2023-03-01 00:41:39+00:00,1630729979058352128,"@puji_setyanto @CH4__15R Eh TOLOL bocah cacat,...",septa_ringga,1,indonesia,eh tolol bocah cacat di situ ada perangkat des...,"[eh, tolol, bocah, cacat, di, situ, ada, peran...","[eh, tolol, bocah, cacat, di, situ, ada, peran...","[eh, tolol, bocah, cacat, situ, perangkat, des...","[eh, tolol, bocah, cacat, situ, perangkat, des...",eh tolol bocah cacat situ perangkat desa langs...
